In [152]:
#Import all the relevant packages
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [153]:
#Set up user defined constants and file info
NCmetadatafile = '220211_2M1F_NaiveChoice_metadata.csv'
FCmetadatafile = '220211_2F1M_final_metadata.csv'
huddle_time = 20 #ms

make_duplicate_plots = True

In [154]:
#Define support functions in this cell
def extract_lane_info(file_list):
    extracted_names = []
    for file in range(0, len(file_list),1):
        components = file_list[file].split('_')
        for l in range(0, len(components),1):
            if "Lane" in components[l]:
                extracted_names.append(components[l])
    return extracted_names
                

def calculate_PPI(datafile):
    #calculate time huddling
    time_huddling_right = []
    time_huddling_left = []
    for i in range(0, len(datafile),1):
        row = datafile.loc[i]
        timecontactright = row.Contact_Right_stop - row.Contact_Right_start
        timecontactleft = row.Contact_Left_stop - row.Contact_Left_start
        if timecontactright >= huddle_time:
            time_huddling_right.append(timecontactright)
        if timecontactleft >= huddle_time:
            time_huddling_left.append(timecontactleft)
    total_huddle_time_right = np.nansum(time_huddling_right)
    total_huddle_time_left = np.nansum(time_huddling_left)
    PPI = (total_huddle_time_right - total_huddle_time_left)/(total_huddle_time_right + total_huddle_time_left)
    return PPI

def calculate_winner(datafile):
    if calculate_PPI(datafile) > 0:
        return 'Right'
    elif calculate_PPI(datafile) == 0:
        return 'None'
    else:
        return 'Left'
    
def plot (datatable):
    return

In [165]:
#Handle the raw data files

# 1 - Generate a list of all the files to work on
NCmetadatatable = pd.read_csv(NCmetadatafile)
FCmetadatatable = pd.read_csv(FCmetadatafile)

data_dir = os.path.join('C:\\','Users','vashv','OneDrive','Documents','GitHub','ppt-lanes','data_files')
#dir_list = os.listdir(data_dir)
#print(dir_list)

NC_all_files = []
NC_all_files += [os.path.join(data_dir, each) for each in os.listdir(data_dir) if 'NaiveChoice' in each]

# 2 - Set up places to store all your eventual calculations
winner_column = []
PPI_column = []

# 3 - Iterate over that list

for file in NC_all_files:
    #Load data
    thisdata = pd.read_csv(file)
    #do all computations by calling functions
    thisPPI = calculate_PPI(thisdata)
    thiswinner = calculate_winner(thisdata)
    #put all of my file calculations in group data table
    PPI_column.append(thisPPI)
    winner_column.append(thiswinner)

# Consolidate all your calculations into one big table

NCtable = NCmetadatatable.drop(6)
NCtable.insert(loc = 0,
              column = 'Lane',
              value = extract_lane_info(NC_all_files))
NCtable['PPI'] = PPI_column
NCtable['Winner'] = winner_column
NCtable = NCtable.drop('File.name', axis = 1)
NCtable.to_csv('Summary_table_NC')

#COmpared to Nerissa - you need to calculate winner and all that for everything, and resave - eh just fix

,Lane,Left,Right,Sex,PPI,Winner
0,Lane1,WT,Oxtr-/-,2M1F,0.974477,Right
1,Lane2,WT,WT,2M1F,0.817165,Right
2,Lane3,Oxtr-/-,WT,2M1F,1.000000,Right
3,Lane4,WT,WT,2M1F,0.134985,Right
4,Lane1,Oxtr-/-,WT,2M1F,0.739965,Right
5,Lane2,WT,WT,2M1F,0.442490,Right
7,Lane4,Oxtr-/-,WT,2M1F,0.909047,Right
8,Lane1,WT,WT,2M1F,-0.651902,Left
9,Lane2,WT,Oxtr-/-,2M1F,0.448250,Right
10,Lane4,WT,Oxtr-/-,2M1F,-0.820928,Left


In [ ]:
#Let's make a function to plot huddling data




sns.set_theme()
sns.relplot(
    data= huddling_time_data, kind = "scatter",
    x="Lane", y="Preference_ratio", #col="time",
    #hue="Left", #style="Left", #size="size",
)